# Run full Granual Predictions

What needs to happen:
- ~~function to temporally and spatially interpolate MERRA2~~
- ~~function to read MERRA2 with interpolation~~
  - PS = surface_pressure
  - T10M = 10-meter_air_temperature (Andy will investigate)
  - TO3 = total_column_ozone
  - TQV = total_precipitable_water_vapor
- ~~function to read VNP02MOD~~
  - in group "observation_data"
    - M14 (for center wavelengths at 8500 nm)
    - M15 (for center wavelengths at 10800 nm)
    - M16 (for center wavelengths at 12000 nm)
- ~~function to read VNP03MOD~~
  - in group "geolocation_data"
    - sensor_azimuth
    - sensor_zenith
    - solar_azimuth
    - solar_zenith
- combine above for model inputs
- check if there are any transformations that need to be made on the inputs before sening them though the model
- load the saved model (see tensorflow.keras.models.load_model as in evaluate.ipynb)
- call the saved model's "predict" method on the combined inputs

## Setup

In [ ]:
from pathlib import Path

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from hot_dust import preprocess, DATADIR

In [ ]:
open_vnp02 = xr.open_dataset(
    "data/granules/VNP02MOD.A2020168.1448.002.2021127084950.nc",
    group="observation_data",
    mask_and_scale=False,
)

# Access the relavant variables
vnp02_variables = open_vnp02[
    [
        "M14",
        "M14_brightness_temperature_lut",
        "M15",
        "M15_brightness_temperature_lut",
        "M16",
        "M16_brightness_temperature_lut",
    ]
]
open_vnp02

# call xr where and fill values less than oe equal to 65527
# xr stack and unstack 3D to 2D

In [ ]:
open_vnp03 = xr.open_dataset(
    "data/granules/VNP03MOD.A2020168.1448.002.2021125194020.nc",
    group="geolocation_data",
)

# Access the relavant variables
vnp03_variables = open_vnp03[
    ["sensor_azimuth", "sensor_zenith", "solar_azimuth", "solar_zenith"]
]

In [ ]:
# Merge the model imputs
vnp02_vnp03 = xr.merge([vnp02_variables, vnp03_variables])
vnp02_vnp03

In [ ]:
# Access the relavant variables
M14_ds = vnp02_vnp03["M14"]  # thermal infared wave length
M14_BTL_ds = vnp02_vnp03["M14_brightness_temperature_lut"]
M15_ds = vnp02_vnp03["M15"]  # thermal infared wave length
M15_BTL_ds = vnp02_vnp03["M15_brightness_temperature_lut"]
M16_ds = vnp02_vnp03["M16"]  # thermal infared wave length
M16_BTL_ds = vnp02_vnp03["M16_brightness_temperature_lut"]

# Convert to brightness temperature in K
M14_scaled = vnp02_vnp03["M14_brightness_temperature_lut"][vnp02_vnp03["M14"]]
valid_min = vnp02_vnp03["M14_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M14_brightness_temperature_lut"].attrs["valid_max"]
M14_scaled = M14_scaled.where((M14_scaled <= valid_max) & (M14_scaled >= valid_min))

M15_scaled = vnp02_vnp03["M15_brightness_temperature_lut"][vnp02_vnp03["M15"]]
valid_min = vnp02_vnp03["M15_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M15_brightness_temperature_lut"].attrs["valid_max"]
M15_scaled = M15_scaled.where((M15_scaled <= valid_max) & (M15_scaled >= valid_min))

M16_scaled = vnp02_vnp03["M16_brightness_temperature_lut"][vnp02_vnp03["M16"]]
valid_min = vnp02_vnp03["M16_brightness_temperature_lut"].attrs["valid_min"]
valid_max = vnp02_vnp03["M16_brightness_temperature_lut"].attrs["valid_max"]
M16_scaled = M16_scaled.where((M16_scaled <= valid_max) & (M16_scaled >= valid_min))

## Extract MERRA-2 Input Variables



## 2D Plot of Input Variables

In [ ]:
# M14 Map
plt.imshow(M14_scaled, cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()  # TODO mask fill values

In [ ]:
# M15 Map
plt.imshow(M15_scaled, cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# M16 Map
plt.imshow(M16_scaled, cmap="jet")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

## 2D Plot of Predicted Dust Optical Thickness

## WIP / Scratch

In [ ]:
xr.open_dataset("data/rt_nn_irdust_training_data.nc")

In [ ]:
xr.open_dataset("data/granules/GMAO_MERRA2.20200616T140000.MET.nc")